#### BNZ Advanced Analytics MLE Technical Exercise - Marcus David Buckland

Code & Responses to the questions found in `Instructions.pdf` below.

In [1]:
import duckdb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
rng = np.random.default_rng(42)

In [2]:
EMPLOYMENT_DATA_FILENAME = 'business-employment-data-march-2024-quarter.zip'
FINANCIAL_DATA_FILENAME = 'business-financial-data-march-2024.zip'

In [3]:
employment_df = pd.read_csv(f'data/{EMPLOYMENT_DATA_FILENAME}')
employment_df.head(3)

,Series_reference,Period,Data_value,Suppressed,STATUS,UNITS,Magnitude,Subject,Group,Series_title_1,Series_title_2,Series_title_3,Series_title_4,Series_title_5
0,BDCQ.SEA1AA,2011.06,80078.0,NaN,F,Number,0,Business Data Collection - BDC,Industry by employment variable,Filled jobs,"Agriculture, Forestry and Fishing",Actual,NaN,NaN
1,BDCQ.SEA1AA,2011.09,78324.0,NaN,F,Number,0,Business Data Collection - BDC,Industry by employment variable,Filled jobs,"Agriculture, Forestry and Fishing",Actual,NaN,NaN
2,BDCQ.SEA1AA,2011.12,85850.0,NaN,F,Number,0,Business Data Collection - BDC,Industry by employment variable,Filled jobs,"Agriculture, Forestry and Fishing",Actual,NaN,NaN


In [4]:
financials_df = pd.read_csv(f'data/{FINANCIAL_DATA_FILENAME}')
financials_df.head(3)

,Series_reference,Period,Data_value,Suppressed,STATUS,UNITS,Magnitude,Subject,Group,Series_title_1,Series_title_2,Series_title_3,Series_title_4,Series_title_5
0,BDCQ.SF1AA2CA,2016.06,1116.386,NaN,F,Dollars,6,Business Data Collection - BDC,Industry by financial variable (NZSIOC Level 2),Sales (operating income),Forestry and Logging,Current prices,Unadjusted,NaN
1,BDCQ.SF1AA2CA,2016.09,1070.874,NaN,F,Dollars,6,Business Data Collection - BDC,Industry by financial variable (NZSIOC Level 2),Sales (operating income),Forestry and Logging,Current prices,Unadjusted,NaN
2,BDCQ.SF1AA2CA,2016.12,1054.408,NaN,F,Dollars,6,Business Data Collection - BDC,Industry by financial variable (NZSIOC Level 2),Sales (operating income),Forestry and Logging,Current prices,Unadjusted,NaN


#### EDA

___

Before getting underway with the five technical questions, it's prudent to have an understanding of both of the datasets that will be utilsed in answering the questions.

These stats.govt.nz website provides the following pages related to these datasets:

1. [Business employment data: March 2024 quarter](https://www.stats.govt.nz/information-releases/business-employment-data-march-2024-quarter/)
1. [Business financial data: March 2024 quarter](https://www.stats.govt.nz/information-releases/business-financial-data-march-2024-quarter/)

<u>The Business employment dataset contains data for 14 tables:</u>

1. Actual employment by industry, filled jobs
1. Seasonally adjusted employment by industry, filled jobs
1. Employment by industry, trend, filled jobs
1. Actual employment by industry, total earnings
1. Actual employment by sex, filled jobs
1. Actual employment by sex, total earnings
1. Actual employment by region, filled jobs
1. Seasonally adjusted employment by region, filled jobs
1. Employment by region, trend, filled jobs
1. Actual employment by region, total earnings
1. Actual employment by territorial authority, filled jobs
1. Actual employment by territorial authority, total earnings
1. Actual employment by age, filled jobs
1. Actual employment by age, total earnings


<u>The Business financial dataset contains data for five tables:</u>

1. Business financial data by industry group - actual sales value							
1. Business financial data by industry group - seasonally adjusted sales value							
1. Business financial data by industry group - actual purchases value							
1. Business financial data by industry group - actual salaries and wages values							
1. Business financial data by industry group - actual operating profit values							

In [5]:
# EDA of Financials Dataset
financials_df.shape # 7635 rows, 14 columns

(7635, 14)

In [6]:
# What are the column names?
[x for x in financials_df.columns]

['Series_reference',
 'Period',
 'Data_value',
 'Suppressed',
 'STATUS',
 'UNITS',
 'Magnitude',
 'Subject',
 'Group',
 'Series_title_1',
 'Series_title_2',
 'Series_title_3',
 'Series_title_4',
 'Series_title_5']

In [7]:
financials_df['Series_reference'].nunique() # 240 unique series references in the dataset

240

In [8]:
# Let's examine the Series_reference column- find 10 random examples
random_row_numbers = rng.integers(0, len(financials_df), size=10)  # 10 random row numbers
series_ref_examples =financials_df.loc[random_row_numbers, 'Series_reference']
series_ref_examples

681     BDCQ.SF1CC3CA
5909     BDCQ.SF3IICA
4997     BDCQ.SF2QQCA
3350     BDCQ.SF1PPCT
3306     BDCQ.SF1PPCS
6555    BDCQ.SF8CC1CA
656     BDCQ.SF1CC2CT
5324    BDCQ.SF3CC2CA
1538     BDCQ.SF1DDCA
719     BDCQ.SF1CC3CS
Name: Series_reference, dtype: object

The series reference can be decomposed- Let's examine in further detail, the first example in our series_ref_examples list: 
* `BDCQ.SF1CC3CA`

* The first 3 chars `BDC` tell us that this dataset is part of the `Business data collection` group.
* The fourth char `Q` tells us that this data is collected on a `Quarterly` basis

Following the full-stop
* There are four categories for the next 3 chars: `SF1`, `SF2`, `SF3`, and finally `SF8`. This corresponds to the variable in the column `Series_title_1`.

* `SF1`= Sales (operating income)
* `SF2`= Purchases and operating expenditure
* `SF3`= Salaries and wages
* `SF8`= Operating profit

The following two chars correspond to the industry type, although it's important to note that it is not an exact 1:1 mapping from char_code : industry type. e.g.

`AA` maps to: Forestry and logging, Fishing, Aquaculture and Agriculture, Forestry and Fishing Support Services, Agriculture, Forestry and Fishing \
`BB` is mining \
`CC` maps to 10 "different" industries contained within the columns- however, the common theme amongst those 10 industries is Manufacturing. \
`DD` is Electricity, Gas, Water and Waste Services. \
`EE` is Construction

The next char futher specifies the industries into their various sub-categories. For example, as mentioned above, `CC` contains manufacturing. If you wish to see each of the industries broken down further, for Manufacturing we see:

| Industry Code | Industry                                                      |
|---------------|---------------------------------------------------------------|
| CC1           | Food, Beverage and Tobacco Product   Manufacturing            |
| CC2           | Textile, Leather, Clothing and Footwear Manufacturing         |
| CC3           | Wood and Paper Products Manufacturing                         |
| CC4           | Printing                                                      |
| CC5           | Petroleum, Chemical, Polymer and Rubber Product Manufacturing |
| CC6           | Non-Metallic Mineral Product Manufacturing                    |
| CC7           | Metal Product Manufacturing                                   |
| CC8           | Transport Equipment, Machinery and Equipment Manufacturing    |
| CC9           | Furniture and Other Manufacturing                             |
| CCC           | Manufacturing                                                 |

The next char is always `C` - My assumption is that it stands for `Current` or `Current prices`.

Finally, the last three chars can be `A`, `S`, and `T` - these correspond to how the data values may have been adjusted.

`A` = Actual \
`S` = Seasonally adjusted \
`T` = Trend


`Period` column - pertains to the Quarter, in the format YYYY.MM. Of note that a machine will interpret this column as a number, rather than a string.

`Data_value` column - the dollar amount in units of $million (see `Magnitude` column)

`Suppressed` column - Can be `Y` or blank. If `Y` then the data_value is suppressed.

`STATUS` column - Can be `C` (Confidential), `F` (Final), or `R` (Revised).

`UNITS` column - always "Dollars" for this dataset. 

`Magnitude` - always 6 for this dataset. Interpret this as "raise 10 to the power of value in this column" & multiply by `Data_value` to get answer in the units found in `UNITS` column.

`Subject` - always "Business Data Collection - BDC" for this dataset.

`Group` - Tells you about the NZIOC Level for the industry, either Level 1 or Level 2. Is worthwhile changing the name of this column as GROUP is a reserved word in SQL and it's going to needlessly complicate our queries (and make them less readable) should we leave it as is. A good candidate to tidy. Initial thoughts are convert column to `NZSIOC_level` and have values of 1 and 2.

`Series_title_x` - Correspond to the grouping variables for the dataset.

___


### Data Tidying

___

In [9]:
# Tidy up the financials_df dataset
display(financials_df.isna().all()) # Series_title_5 contains only NaN values. Let's remove that column
financials_df.drop('Series_title_5', axis=1, inplace=True)
financials_df.head(2)

Series_reference    False
Period              False
Data_value          False
Suppressed          False
STATUS              False
UNITS               False
Magnitude           False
Subject             False
Group               False
Series_title_1      False
Series_title_2      False
Series_title_3      False
Series_title_4      False
Series_title_5       True
dtype: bool

,Series_reference,Period,Data_value,Suppressed,STATUS,UNITS,Magnitude,Subject,Group,Series_title_1,Series_title_2,Series_title_3,Series_title_4
0,BDCQ.SF1AA2CA,2016.06,1116.386,NaN,F,Dollars,6,Business Data Collection - BDC,Industry by financial variable (NZSIOC Level 2),Sales (operating income),Forestry and Logging,Current prices,Unadjusted
1,BDCQ.SF1AA2CA,2016.09,1070.874,NaN,F,Dollars,6,Business Data Collection - BDC,Industry by financial variable (NZSIOC Level 2),Sales (operating income),Forestry and Logging,Current prices,Unadjusted


In [10]:
# The group column contains data relating to the NZSIOC Level
financials_df['Group'].unique() 

array(['Industry by financial variable (NZSIOC Level 2)',
       'Industry by financial variable (NZSIOC Level 1)'], dtype=object)

In [11]:
# Renaming the Group column to NZIOC_Level
financials_df['Group'] = financials_df['Group'].apply(lambda x: 1 if '1' in x else 2)
financials_df.rename({'Group':'NZIOC_Level'}, axis=1, inplace=True)
financials_df.head(2)

,Series_reference,Period,Data_value,Suppressed,STATUS,UNITS,Magnitude,Subject,NZIOC_Level,Series_title_1,Series_title_2,Series_title_3,Series_title_4
0,BDCQ.SF1AA2CA,2016.06,1116.386,NaN,F,Dollars,6,Business Data Collection - BDC,2,Sales (operating income),Forestry and Logging,Current prices,Unadjusted
1,BDCQ.SF1AA2CA,2016.09,1070.874,NaN,F,Dollars,6,Business Data Collection - BDC,2,Sales (operating income),Forestry and Logging,Current prices,Unadjusted


In [12]:
# Subject colum is only ever 'Business Data Collection - BDC'. It can be removed.
display(financials_df['Subject'].unique())
financials_df.drop('Subject', axis=1, inplace=True)
financials_df.head(2)

array(['Business Data Collection - BDC'], dtype=object)

,Series_reference,Period,Data_value,Suppressed,STATUS,UNITS,Magnitude,NZIOC_Level,Series_title_1,Series_title_2,Series_title_3,Series_title_4
0,BDCQ.SF1AA2CA,2016.06,1116.386,NaN,F,Dollars,6,2,Sales (operating income),Forestry and Logging,Current prices,Unadjusted
1,BDCQ.SF1AA2CA,2016.09,1070.874,NaN,F,Dollars,6,2,Sales (operating income),Forestry and Logging,Current prices,Unadjusted


In [13]:
# Series_title_2 pertains to industry type. Let's rename this column
display(financials_df['Series_title_2'].unique())
financials_df.rename({'Series_title_2':'Industry_Type'}, axis=1, inplace=True)
financials_df.head(2)

array(['Forestry and Logging',
       'Fishing, Aquaculture and Agriculture, Forestry and Fishing Support Services',
       'Agriculture, Forestry and Fishing', 'Mining',
       'Food, Beverage and Tobacco Product Manufacturing',
       'Textile, Leather, Clothing and Footwear Manufacturing',
       'Wood and Paper Products Manufacturing', 'Printing',
       'Petroleum, Chemical, Polymer and Rubber Product Manufacturing',
       'Non-Metallic Mineral Product Manufacturing',
       'Metal Product Manufacturing',
       'Transport Equipment, Machinery and Equipment Manufacturing',
       'Furniture and Other Manufacturing', 'Manufacturing',
       'Electricity, Gas, Water and Waste Services', 'Construction',
       'Wholesale Trade', 'Retail Trade',
       'Accommodation and Food Services',
       'Retail Trade and Accommodation',
       'Transport, Postal and Warehousing',
       'Information Media and Telecommunications',
       'Rental, Hiring and Real Estate Services',
       'Profes

,Series_reference,Period,Data_value,Suppressed,STATUS,UNITS,Magnitude,NZIOC_Level,Series_title_1,Industry_Type,Series_title_3,Series_title_4
0,BDCQ.SF1AA2CA,2016.06,1116.386,NaN,F,Dollars,6,2,Sales (operating income),Forestry and Logging,Current prices,Unadjusted
1,BDCQ.SF1AA2CA,2016.09,1070.874,NaN,F,Dollars,6,2,Sales (operating income),Forestry and Logging,Current prices,Unadjusted


In [14]:
# Series_title_3 contains both 'Current' and 'Current prices'- There is no distinction between this and we can simplify
# Confirmed with Reuben Harrison at Stats NZ
display(financials_df['Series_title_3'].unique())

financials_df['Series_title_3'] = financials_df['Series_title_3'].apply(lambda x: 'Current' if x == 'Current prices' else x)
financials_df.rename({'Series_title_3':'Price'}, axis=1, inplace=True)
financials_df['Price'].unique()

array(['Current prices', 'Current'], dtype=object)

array(['Current'], dtype=object)

In [15]:
# Series_title_4 pertains to if an adjustment has been made to the data value
display(financials_df['Series_title_4'].unique())

financials_df.rename({'Series_title_4':'Adjustment'}, axis=1, inplace=True)
financials_df.head(2)

array(['Unadjusted', 'Seasonally adjusted', 'Trend'], dtype=object)

,Series_reference,Period,Data_value,Suppressed,STATUS,UNITS,Magnitude,NZIOC_Level,Series_title_1,Industry_Type,Price,Adjustment
0,BDCQ.SF1AA2CA,2016.06,1116.386,NaN,F,Dollars,6,2,Sales (operating income),Forestry and Logging,Current,Unadjusted
1,BDCQ.SF1AA2CA,2016.09,1070.874,NaN,F,Dollars,6,2,Sales (operating income),Forestry and Logging,Current,Unadjusted


In [16]:
# Finally- we can create separate dataframes- the dataset is nicely partitioned already by Series_title_1
display(financials_df['Series_title_1'].unique())

sales_and_operating_income_df = financials_df[financials_df.loc[:, 'Series_title_1'] == 'Sales (operating income)'].copy()
purchases_and_operating_expenditure_df = financials_df[financials_df.loc[:, 'Series_title_1'] == 'Purchases and operating expenditure'].copy()
salaries_and_wages_df = financials_df[financials_df.loc[:, 'Series_title_1'] == 'Salaries and wages'].copy()
operating_profit_df = financials_df[financials_df.loc[:, 'Series_title_1'] == 'Operating profit'].copy()

# We can now drop the Series_title_1 column for each of the newly created dataframes as it is redundant.
sales_and_operating_income_df.drop('Series_title_1', axis=1, inplace=True)
purchases_and_operating_expenditure_df.drop('Series_title_1', axis=1, inplace=True)
salaries_and_wages_df.drop('Series_title_1', axis=1, inplace=True)
operating_profit_df.drop('Series_title_1', axis=1, inplace=True)

array(['Sales (operating income)', 'Purchases and operating expenditure',
       'Salaries and wages', 'Operating profit'], dtype=object)

In [17]:
# Tidy up the employment_df dataset
employment_df.isna().all() # Series_title_4 & Series_title_5 contains only NaN values. Let's remove those columns
employment_df.drop(['Series_title_4', 'Series_title_5'], axis=1, inplace=True)
employment_df.head(2)

,Series_reference,Period,Data_value,Suppressed,STATUS,UNITS,Magnitude,Subject,Group,Series_title_1,Series_title_2,Series_title_3
0,BDCQ.SEA1AA,2011.06,80078.0,NaN,F,Number,0,Business Data Collection - BDC,Industry by employment variable,Filled jobs,"Agriculture, Forestry and Fishing",Actual
1,BDCQ.SEA1AA,2011.09,78324.0,NaN,F,Number,0,Business Data Collection - BDC,Industry by employment variable,Filled jobs,"Agriculture, Forestry and Fishing",Actual


In [18]:
# Subject colum is only ever 'Business Data Collection - BDC'. It can be removed.
display(employment_df['Subject'].unique())
employment_df.drop('Subject', axis=1, inplace=True)
employment_df.head(2)

array(['Business Data Collection - BDC'], dtype=object)

,Series_reference,Period,Data_value,Suppressed,STATUS,UNITS,Magnitude,Group,Series_title_1,Series_title_2,Series_title_3
0,BDCQ.SEA1AA,2011.06,80078.0,NaN,F,Number,0,Industry by employment variable,Filled jobs,"Agriculture, Forestry and Fishing",Actual
1,BDCQ.SEA1AA,2011.09,78324.0,NaN,F,Number,0,Industry by employment variable,Filled jobs,"Agriculture, Forestry and Fishing",Actual


In [19]:
# Series_title_3 pertains to if an ajustment has been made to the data value
display(employment_df['Series_title_3'].unique())

employment_df.rename({'Series_title_3':'Adjustment'}, axis=1, inplace=True)
employment_df.head(2)

array(['Actual', 'Seasonally adjusted', 'Trend'], dtype=object)

,Series_reference,Period,Data_value,Suppressed,STATUS,UNITS,Magnitude,Group,Series_title_1,Series_title_2,Adjustment
0,BDCQ.SEA1AA,2011.06,80078.0,NaN,F,Number,0,Industry by employment variable,Filled jobs,"Agriculture, Forestry and Fishing",Actual
1,BDCQ.SEA1AA,2011.09,78324.0,NaN,F,Number,0,Industry by employment variable,Filled jobs,"Agriculture, Forestry and Fishing",Actual


In [20]:
# As we did above, we can split up the employment dataframe into smaller dataframes.
# This time, let's only create the dataframes required for answering Q1-Q3 in Instructions.pdf

# Q1 requires 'filled jobs by industry'
filled_jobs_by_industry_df = employment_df[(employment_df['Series_title_1'] == 'Filled jobs') & (employment_df['Group'] == 'Industry by employment variable')].copy()
display(filled_jobs_by_industry_df.head(2))

# Q2 requires only data from the financials dataset, which has already been tidied.

# Q3 requires 'filled jobs by territorial authority'
filled_jobs_by_territorial_authority_df = employment_df[(employment_df['Series_title_1'] == 'Filled jobs') & (employment_df['Group'] == 'Territorial authority by employment variable')].copy()
filled_jobs_by_territorial_authority_df.head(2)

,Series_reference,Period,Data_value,Suppressed,STATUS,UNITS,Magnitude,Group,Series_title_1,Series_title_2,Adjustment
0,BDCQ.SEA1AA,2011.06,80078.0,NaN,F,Number,0,Industry by employment variable,Filled jobs,"Agriculture, Forestry and Fishing",Actual
1,BDCQ.SEA1AA,2011.09,78324.0,NaN,F,Number,0,Industry by employment variable,Filled jobs,"Agriculture, Forestry and Fishing",Actual


,Series_reference,Period,Data_value,Suppressed,STATUS,UNITS,Magnitude,Group,Series_title_1,Series_title_2,Adjustment
12480,BDCQ.SEE1001A,2011.06,17061.0,NaN,F,Number,0,Territorial authority by employment variable,Filled jobs,Far North District,Actual
12481,BDCQ.SEE1001A,2011.09,16702.0,NaN,F,Number,0,Territorial authority by employment variable,Filled jobs,Far North District,Actual


In [21]:
# Tidying filled_jobs_by_industry_df
# We can remove the Group column and the Series_title_1 column from this dataframe
display(filled_jobs_by_industry_df['Group'].unique()) # Only ever Industry by employment variable
display(filled_jobs_by_industry_df['Series_title_1'].unique()) # Only ever Filled jobs

filled_jobs_by_industry_df.drop(['Group', 'Series_title_1'], axis=1, inplace=True)
filled_jobs_by_industry_df.head(2)

array(['Industry by employment variable'], dtype=object)

array(['Filled jobs'], dtype=object)

,Series_reference,Period,Data_value,Suppressed,STATUS,UNITS,Magnitude,Series_title_2,Adjustment
0,BDCQ.SEA1AA,2011.06,80078.0,NaN,F,Number,0,"Agriculture, Forestry and Fishing",Actual
1,BDCQ.SEA1AA,2011.09,78324.0,NaN,F,Number,0,"Agriculture, Forestry and Fishing",Actual


In [22]:
# Finally- let's rename Series_title 2 to something more meaningful
filled_jobs_by_industry_df.rename({'Series_title_2':'Industry_Type'}, axis=1, inplace=True)
filled_jobs_by_industry_df.head(2)

,Series_reference,Period,Data_value,Suppressed,STATUS,UNITS,Magnitude,Industry_Type,Adjustment
0,BDCQ.SEA1AA,2011.06,80078.0,NaN,F,Number,0,"Agriculture, Forestry and Fishing",Actual
1,BDCQ.SEA1AA,2011.09,78324.0,NaN,F,Number,0,"Agriculture, Forestry and Fishing",Actual


In [23]:
# Tidying filled_jobs_by_industry_df
# We can remove the Group column and the Series_title_1 column from this dataframe
filled_jobs_by_territorial_authority_df.drop(['Group', 'Series_title_1'], axis=1, inplace=True)
filled_jobs_by_territorial_authority_df.head(2)

,Series_reference,Period,Data_value,Suppressed,STATUS,UNITS,Magnitude,Series_title_2,Adjustment
12480,BDCQ.SEE1001A,2011.06,17061.0,NaN,F,Number,0,Far North District,Actual
12481,BDCQ.SEE1001A,2011.09,16702.0,NaN,F,Number,0,Far North District,Actual


In [24]:
# Finally- let's rename Series_title 2 to something more meaningful
filled_jobs_by_territorial_authority_df.rename({'Series_title_2':'Territorial_Authority'}, axis=1, inplace=True)
filled_jobs_by_territorial_authority_df.head(2)

,Series_reference,Period,Data_value,Suppressed,STATUS,UNITS,Magnitude,Territorial_Authority,Adjustment
12480,BDCQ.SEE1001A,2011.06,17061.0,NaN,F,Number,0,Far North District,Actual
12481,BDCQ.SEE1001A,2011.09,16702.0,NaN,F,Number,0,Far North District,Actual


### Create the SQL Tables

___

In [25]:
# salaries_and_wages table
duckdb.sql("""
        CREATE TABLE salaries_and_wages
        AS SELECT *
        FROM salaries_and_wages_df
        """)

# employment_by_industry table
duckdb.sql("""
        CREATE TABLE employment_by_industry
        AS SELECT *
        FROM filled_jobs_by_industry_df
        """)

# employment_by_territorial_authority table
duckdb.sql("""
        CREATE TABLE employment_by_territorial_authority
        AS SELECT *
        FROM filled_jobs_by_territorial_authority_df
        """)

In [26]:
duckdb.sql("""
        SELECT * 
        FROM salaries_and_wages
        LIMIT 5
        """)

┌──────────────────┬─────────┬────────────┬────────────┬─────────┬─────────┬───────────┬─────────────┬──────────────────────┬─────────┬────────────┐
│ Series_reference │ Period  │ Data_value │ Suppressed │ STATUS  │  UNITS  │ Magnitude │ NZIOC_Level │    Industry_Type     │  Price  │ Adjustment │
│     varchar      │ double  │   double   │   int32    │ varchar │ varchar │   int64   │    int64    │       varchar        │ varchar │  varchar   │
├──────────────────┼─────────┼────────────┼────────────┼─────────┼─────────┼───────────┼─────────────┼──────────────────────┼─────────┼────────────┤
│ BDCQ.SF3AA2CA    │ 2016.06 │     65.087 │       NULL │ F       │ Dollars │         6 │           2 │ Forestry and Logging │ Current │ Unadjusted │
│ BDCQ.SF3AA2CA    │ 2016.09 │     66.088 │       NULL │ F       │ Dollars │         6 │           2 │ Forestry and Logging │ Current │ Unadjusted │
│ BDCQ.SF3AA2CA    │ 2016.12 │     72.872 │       NULL │ F       │ Dollars │         6 │           2 │ For

In [27]:
duckdb.sql("""
        SELECT * 
        FROM employment_by_industry
        LIMIT 5
        """)

┌──────────────────┬─────────┬────────────┬────────────┬─────────┬─────────┬───────────┬───────────────────────────────────┬────────────┐
│ Series_reference │ Period  │ Data_value │ Suppressed │ STATUS  │  UNITS  │ Magnitude │           Industry_Type           │ Adjustment │
│     varchar      │ double  │   double   │   int32    │ varchar │ varchar │   int64   │              varchar              │  varchar   │
├──────────────────┼─────────┼────────────┼────────────┼─────────┼─────────┼───────────┼───────────────────────────────────┼────────────┤
│ BDCQ.SEA1AA      │ 2011.06 │    80078.0 │       NULL │ F       │ Number  │         0 │ Agriculture, Forestry and Fishing │ Actual     │
│ BDCQ.SEA1AA      │ 2011.09 │    78324.0 │       NULL │ F       │ Number  │         0 │ Agriculture, Forestry and Fishing │ Actual     │
│ BDCQ.SEA1AA      │ 2011.12 │    85850.0 │       NULL │ F       │ Number  │         0 │ Agriculture, Forestry and Fishing │ Actual     │
│ BDCQ.SEA1AA      │ 2012.03 │    

In [28]:
duckdb.sql("""
        SELECT * 
        FROM employment_by_territorial_authority
        LIMIT 5
        """)

┌──────────────────┬─────────┬────────────┬────────────┬─────────┬─────────┬───────────┬───────────────────────┬────────────┐
│ Series_reference │ Period  │ Data_value │ Suppressed │ STATUS  │  UNITS  │ Magnitude │ Territorial_Authority │ Adjustment │
│     varchar      │ double  │   double   │  varchar   │ varchar │ varchar │   int64   │        varchar        │  varchar   │
├──────────────────┼─────────┼────────────┼────────────┼─────────┼─────────┼───────────┼───────────────────────┼────────────┤
│ BDCQ.SEE1001A    │ 2011.06 │    17061.0 │ NULL       │ F       │ Number  │         0 │ Far North District    │ Actual     │
│ BDCQ.SEE1001A    │ 2011.09 │    16702.0 │ NULL       │ F       │ Number  │         0 │ Far North District    │ Actual     │
│ BDCQ.SEE1001A    │ 2011.12 │    17008.0 │ NULL       │ F       │ Number  │         0 │ Far North District    │ Actual     │
│ BDCQ.SEE1001A    │ 2012.03 │    16630.0 │ NULL       │ F       │ Number  │         0 │ Far North District    │ Actua

#### Question 1

___

Of the industries where salaries and wages data did NOT exist in 2016 and only appeared later, which industry had the highest average value for actual filled jobs across time and what was that value?

#### Question 1 Answer

Of the industries where salaries and wages data did not exist in 2016, 'Retail Trade' was the industry with the highest average value for actual filled jobs across time with and average number of Actual filled jobs of 194,000 (rounded to the nearest thousand.)

In [29]:
# Question 1 - using DuckDB SQL query
duckdb.sql("""
    WITH industries_of_interest AS (
        SELECT industry_type, MIN(period) as earliest_period
        FROM salaries_and_wages
        GROUP BY industry_type
        HAVING earliest_period > 2016.13
    )
    SELECT industry_type, AVG(data_value) as avg_filled_jobs
    FROM employment_by_industry
    WHERE Adjustment = 'Actual'
    GROUP BY industry_type
    HAVING industry_type IN (SELECT industry_type FROM industries_of_interest)
    ORDER BY avg_filled_jobs DESC
    LIMIT 1;
""")

┌───────────────┬────────────────────┐
│ Industry_Type │  avg_filled_jobs   │
│    varchar    │       double       │
├───────────────┼────────────────────┤
│ Retail Trade  │ 194053.71153846153 │
└───────────────┴────────────────────┘

In [30]:
# Before jumping into the sql query- let's first work through the problem iteratively
# This process will help us break down the problem into smaller chunks and also
# give us confidence in our sql query result (if they match!)

df = pd.DataFrame(salaries_and_wages_df.groupby('Industry_Type')['Period'].agg("min").sort_values(ascending=False))
df = df[df['Period'] > 2016.13]

# The three industries which only appear in the salaries and wages dataset after 2016:
[x for x in df.index]

['Accommodation and Food Services',
 'Retail Trade and Accommodation',
 'Retail Trade']

In [31]:
# Filter the filled_jobs_by_industry_df
df2 = filled_jobs_by_industry_df[(filled_jobs_by_industry_df['Industry_Type'].isin(df.index)) & (filled_jobs_by_industry_df['Adjustment'] == 'Actual')]

# We note 'Retail Trade and Accommodation' is not included as an industry type in the employment dataset but is included in the salaries and wages dataset
display(df2['Industry_Type'].unique()) 

q1_ans = pd.DataFrame(df2.groupby('Industry_Type')['Data_value'].agg("mean")) 
q1_ans # 194,000 filled jobs for Retail Trade

array(['Retail Trade', 'Accommodation and Food Services'], dtype=object)

,Data_value
Industry_Type,
Accommodation and Food Services,130958.942308
Retail Trade,194053.711538


In [32]:
# Of the industries where salaries and wages data did not exist in 2016, 'Retail Trade' was the industry with the highest average value for actual filled jobs across time with 
# and average number of Actual filled jobs of 194,000 (rounded to the nearest thousand.)
pd.DataFrame(q1_ans.sort_values('Data_value', ascending=False)).rename({'Data_value':"Actual_filled_jobs"}, axis=1).head(1)

,Actual_filled_jobs
Industry_Type,
Retail Trade,194053.711538


#### Question 2

___

Provide the answer and write a DuckDB SQL query to show which year/month combination and industry had the second highest seasonally adjusted operating income sales across all the business industries in New Zealand that are categorised as NZSIOC level 2.


#### Question 2 Answer

The year/month combination and inudstry that had the second highest seasonally adjusted operating income sales across all the business industries categorised as NZSIOC level 2 was `2023.03` and `Wholesale Trade` which had Seasonally adjusted Sales and Operating Income of NZD $38,810,022,000

In [33]:
# Question 2 with DuckDB SQL query
duckdb.sql("""
        SELECT Period, Industry_Type, Data_value as sales_and_operating_income
        FROM sales_and_operating_income_df
        WHERE Adjustment = 'Seasonally adjusted'
        AND NZIOC_Level = 2
        ORDER BY Data_value DESC
        LIMIT 1 OFFSET 1
        """)

┌─────────┬─────────────────┬────────────────────────────┐
│ Period  │  Industry_Type  │ sales_and_operating_income │
│ double  │     varchar     │           double           │
├─────────┼─────────────────┼────────────────────────────┤
│ 2023.03 │ Wholesale Trade │                  38810.022 │
└─────────┴─────────────────┴────────────────────────────┘

In [34]:
# Q2 with Python
# Filter the sales_and_operating_income dataframe to include only Seasonally Adjusted values
df = sales_and_operating_income_df[(sales_and_operating_income_df['Adjustment'] == 'Seasonally adjusted') & (sales_and_operating_income_df['NZIOC_Level'] == 2)].copy()

q2_ans = df.sort_values('Data_value', ascending=False).loc[:, ['Period', 'Industry_Type', 'Data_value']].iloc[[1]]
q2_ans # 2023.09, 'Wholesale Trade

,Period,Industry_Type,Data_value
1883,2023.03,Wholesale Trade,38810.022


#### Question 3
Create a DuckDB SQL query to calculate the quarterly cumulative number of filled jobs over time for the territorial authority with the highest average value of filled jobs across time. You may not use window functions in your query.

#### Question 3 Answer

Answer goes here...

In [35]:
# Q3 python
# Territorial Authority dataset does not have `Seasonally Adjusted` or `Trend` for filled jobs- only `Actual`
filled_jobs_by_territorial_authority_df.Adjustment.unique()

array(['Actual'], dtype=object)

In [36]:
# What Territorial Authority has the highest average number of filled jobs?
highest_avg_filled_jobs_ta = filled_jobs_by_territorial_authority_df.groupby('Territorial_Authority')['Data_value'].agg("mean").idxmax() # Auckland
highest_avg_filled_jobs_ta 

'Auckland'

In [37]:
df = filled_jobs_by_territorial_authority_df[filled_jobs_by_territorial_authority_df['Territorial_Authority'] == highest_avg_filled_jobs_ta].sort_values('Period')

# Create a new column called cumulative_filled_jobs which is a cumulative sum of Data_value column
df['cumulative_filled_jobs'] = df['Data_value'].cumsum()
display(df.head(5))
display(df.tail(5))

,Series_reference,Period,Data_value,Suppressed,STATUS,UNITS,Magnitude,Territorial_Authority,Adjustment,cumulative_filled_jobs
15912,BDCQ.SEE1076A,2011.06,558627.0,NaN,F,Number,0,Auckland,Actual,558627.0
15913,BDCQ.SEE1076A,2011.09,565439.0,NaN,F,Number,0,Auckland,Actual,1124066.0
15914,BDCQ.SEE1076A,2011.12,573729.0,NaN,F,Number,0,Auckland,Actual,1697795.0
15915,BDCQ.SEE1076A,2012.03,560373.0,NaN,F,Number,0,Auckland,Actual,2258168.0
15916,BDCQ.SEE1076A,2012.06,572938.0,NaN,F,Number,0,Auckland,Actual,2831106.0


,Series_reference,Period,Data_value,Suppressed,STATUS,UNITS,Magnitude,Territorial_Authority,Adjustment,cumulative_filled_jobs
15959,BDCQ.SEE1076A,2023.03,766435.0,NaN,F,Number,0,Auckland,Actual,31847062.0
15960,BDCQ.SEE1076A,2023.06,782096.0,NaN,F,Number,0,Auckland,Actual,32629158.0
15961,BDCQ.SEE1076A,2023.09,782639.0,NaN,F,Number,0,Auckland,Actual,33411797.0
15962,BDCQ.SEE1076A,2023.12,795680.0,NaN,F,Number,0,Auckland,Actual,34207477.0
15963,BDCQ.SEE1076A,2024.03,787534.0,NaN,F,Number,0,Auckland,Actual,34995011.0


In [38]:
# DuckDB SQL query to answer question 3 - currently using OVER which is a window function. :(
duckdb.sql("""
        WITH highest_avg_filled_jobs_ta AS (
            SELECT Territorial_Authority
            FROM employment_by_territorial_authority
            GROUP BY Territorial_Authority
            ORDER BY AVG(Data_value) DESC
            LIMIT 1
),
        cumulative_filled_jobs AS (
            SELECT Period, Territorial_Authority, Data_value, SUM(Data_value) OVER (PARTITION BY Territorial_Authority ORDER BY Period) AS cumulative_filled_jobs
            FROM employment_by_territorial_authority
            WHERE Territorial_Authority IN (SELECT Territorial_Authority FROM highest_avg_filled_jobs_ta)
)
        SELECT Period, Territorial_Authority, cumulative_filled_jobs
        FROM cumulative_filled_jobs
        ORDER BY Period ASC;
        """)

┌─────────┬───────────────────────┬────────────────────────┐
│ Period  │ Territorial_Authority │ cumulative_filled_jobs │
│ double  │        varchar        │         double         │
├─────────┼───────────────────────┼────────────────────────┤
│ 2011.06 │ Auckland              │               558627.0 │
│ 2011.09 │ Auckland              │              1124066.0 │
│ 2011.12 │ Auckland              │              1697795.0 │
│ 2012.03 │ Auckland              │              2258168.0 │
│ 2012.06 │ Auckland              │              2831106.0 │
│ 2012.09 │ Auckland              │              3401783.0 │
│ 2012.12 │ Auckland              │              3981507.0 │
│ 2013.03 │ Auckland              │              4556314.0 │
│ 2013.06 │ Auckland              │              5145225.0 │
│ 2013.09 │ Auckland              │              5731581.0 │
│    ·    │    ·                  │                  ·     │
│    ·    │    ·                  │                  ·     │
│    ·    │    ·        